<a href="https://colab.research.google.com/github/VaibhavSingh1311/Syngas-Prediction-from-Biomass-FeedStock/blob/main/Hybrid%20Model%2B%20Web%20Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import warnings
from scipy.optimize import differential_evolution

# Install required packages
!pip install xgboost -q

# Import machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline

# Import additional ML libraries
import xgboost as xgb
import lightgbm as lgb
import joblib

# Import visualization libraries
import missingno as msno

# Suppress warnings
warnings.filterwarnings('ignore')

# Set style for plots
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (8, 4)

# Configuration
RANDOM_STATE = 42
SAVE_DIR = "/content/drive/MyDrive/ML_Lab_Datasets/"
os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:
# Block-2

# Public Drive link of the Dataset(EXPERIMENTAL(103Rows)
# https://drive.google.com/file/d/1OJe6akOYU120p9PPCD5HA4CjUusvABmd/view?usp=sharing

# Public Drive link of the Dataset(SYNTHETIC(1000Rows)
# https://drive.google.com/file/d/1YMqV6IqDZ-YtTBb7bjjfJ-sszxJ5xOUr/view?usp=drive_link


# Load Dataset
file_path = "/content/drive/MyDrive/ML_Lab_Datasets/final_synthetic_biomass_dataset.csv"
df = pd.read_csv(file_path)

print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")


In [ ]:
# Data cleaning and preparation

print("\n" + "="*50)
print("DATA CLEANING")
print("="*50)

# Clean column names and remove duplicates
df.columns = df.columns.str.strip()
df.drop_duplicates(inplace=True)

# Convert columns to numeric
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

# Replace empty values with NaN
df.replace(['', ' ', '-', '—', 'nan', 'NaN'], np.nan, inplace=True)

print("Missing values:")
print(df.isnull().sum())

print("\nDataset head:")
print(df.head())

# Define target variables
target_cols = ['H2', 'CO', 'CO2', 'CH4']

print("\nTarget variables check:")
for target in target_cols:
    if target in df.columns:
        count = df[target].notna().sum()
        print(f"{target}: {count} values")

# Remove rows with all targets missing
df_clean = df.dropna(subset=target_cols, how='all')
print(f"\nDataset after cleaning: {df_clean.shape}")

# Split features and targets
X = df_clean.drop(columns=target_cols, errors='ignore')
y = df_clean[target_cols]

print(f"Features: {X.shape}, Targets: {y.shape}")

# Identify column types
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Numeric columns: {len(numeric_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

# Train-test split
print("\nTrain-test split...")

if 'TYPES' in X.columns:
    # Check if we can stratify by biomass type
    type_counts = X['TYPES'].value_counts()
    if (type_counts >= 2).all():
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=X['TYPES']
        )
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Remove rows with missing targets
train_nan = y_train.isna().any(axis=1).sum()
test_nan = y_test.isna().any(axis=1).sum()

print(f"\nRows with missing targets - Train: {train_nan}, Test: {test_nan}")

X_train = X_train[~y_train.isna().any(axis=1)]
y_train = y_train[~y_train.isna().any(axis=1)]
X_test = X_test[~y_test.isna().any(axis=1)]
y_test = y_test[~y_test.isna().any(axis=1)]

print(f"Final training set: {X_train.shape}")
print(f"Final test set: {X_test.shape}")



In [ ]:


# ===============================================
# COMBINED MODEL TRAINING - ALL MODELS TOGETHER
# ===============================================
print("\n" + "="*60)
print("TRAINING ALL MODELS TOGETHER")
print("="*60)

# Check dataset size
print(f"  Dataset Info:")
print(f"  Training samples: {X_train.shape[0]}")
print(f"  Test samples: {X_test.shape[0]}")
print(f"  Features: {X_train.shape[1]}")
print(f"  Targets: {y_train.shape[1]}")

# Set cross-validation folds based on dataset size
if X_train.shape[0] < 100:
    cv_folds = min(3, X_train.shape[0] // 10)
    print(f"Small dataset - using {cv_folds}-fold CV")
else:
    cv_folds = 5
    print(f"Using {cv_folds}-fold cross-validation")

# Import additional models if needed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, VotingRegressor

In [ ]:
# Define preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

print("Preprocessor defined successfully!")

In [ ]:
# Define All Models with Hyperparameters

print("\n" + "="*60)
print("DEFINING MODELS WITH HYPERPARAMETERS")
print("="*60)

# Initialize models dictionary
models = {}

# 1. RANDOM FOREST MODEL
models['RandomForest_Tuned'] = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(
        RandomForestRegressor(
            n_estimators=200,
            random_state=42,
            n_jobs=-1,
            max_depth=15,
            min_samples_split=3,
            min_samples_leaf=1,
            max_features=0.7,
            min_weight_fraction_leaf=0.0,
            max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            bootstrap=True,
            oob_score=True
        )
    ))
])

# 2. XGBOOST MODEL
try:
    import xgboost as xgb
    models['XGBoost'] = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', MultiOutputRegressor(
            xgb.XGBRegressor(
                n_estimators=500,
                max_depth=8,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.5,
                reg_lambda=0.5,
                random_state=42,
                n_jobs=-1,
                eval_metric='rmse'
            )
        ))
    ])
except ImportError:
    print("XGBoost not available, skipping...")

# 3. LIGHTGBM MODEL
try:
    import lightgbm as lgb
    models['LightGBM'] = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', MultiOutputRegressor(
            lgb.LGBMRegressor(
                n_estimators=300,
                max_depth=5,
                learning_rate=0.01,
                num_leaves=31,
                subsample=0.7,
                colsample_bytree=0.7,
                reg_alpha=1.0,
                reg_lambda=1.0,
                min_child_samples=10,
                random_state=42,
                n_jobs=-1,
                verbose=-1
            )
        ))
    ])
except ImportError:
    print("LightGBM not available, skipping...")

# 4. ANN MODEL
models['ANN'] = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(
        MLPRegressor(
            hidden_layer_sizes=(128, 64, 32),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            batch_size=32,
            learning_rate='adaptive',
            learning_rate_init=0.001,
            max_iter=2000,
            random_state=42,
            early_stopping=True,
            validation_fraction=0.2,
            n_iter_no_change=20,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-08,
            verbose=False
        )
    ))
])

# 5. SVR MODEL
models['SVR'] = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(
        make_pipeline(
            StandardScaler(),
            SVR(
                kernel='rbf',
                C=10.0,
                epsilon=0.01,
                gamma='scale',
                max_iter=50000,
                cache_size=1000,
                shrinking=True,
                tol=0.001
            )
        )
    ))
])

print(f"Models defined: {len(models)}")
print("Model names:", list(models.keys()))

In [ ]:
# Train All Models
import time
from sklearn.model_selection import KFold, cross_val_score

print("TRAINING ALL MODELS")


# Initialize tracking variables
all_model_results = {}
best_model = None
best_model_name = ""
best_test_score = -np.inf

# Create KFold for cross-validation
kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)

# Train all models
for name, model in models.items():
    print(f"\n" + "="*40)
    print(f"Training: {name}")
    print("="*40)

    try:
        start_time = time.time()

        # Cross-validation
        cv_scores = cross_val_score(model, X_train, y_train,
                                  cv=kf,
                                  scoring='r2',
                                  n_jobs=-1)

        mean_cv_score = cv_scores.mean()
        std_cv_score = cv_scores.std()

        # Train the model
        model.fit(X_train, y_train)

        # Training predictions
        y_train_pred = model.predict(X_train)
        train_r2 = r2_score(y_train, y_train_pred, multioutput='uniform_average')

        # Test predictions
        y_test_pred = model.predict(X_test)
        test_r2 = r2_score(y_test, y_test_pred, multioutput='uniform_average')

        # Calculate metrics
        generalization_gap = train_r2 - test_r2
        training_time = time.time() - start_time

        # Individual target performance
        target_performance = {}
        for i, target in enumerate(target_cols):
            if i < y_test_pred.shape[1] and target in y_test.columns:
                target_r2 = r2_score(y_test[target], y_test_pred[:, i])
                target_mae = mean_absolute_error(y_test[target], y_test_pred[:, i])
                target_performance[target] = {'R2': target_r2, 'MAE': target_mae}

        # Store results
        all_model_results[name] = {
            'cv_score': mean_cv_score,
            'cv_std': std_cv_score,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'generalization_gap': generalization_gap,
            'training_time': training_time,
            'target_performance': target_performance,
            'model': model
        }

        # Print results
        print(f"CV R²: {mean_cv_score:.4f} (±{std_cv_score:.4f})")
        print(f"Training R²: {train_r2:.4f}")
        print(f"Test R²: {test_r2:.4f}")
        print(f"Generalization gap: {generalization_gap:.4f}")
        print(f"Training time: {training_time:.2f} seconds")

        # Show individual target performance
        if target_performance:
            print("Individual target performance:")
            for target, perf in target_performance.items():
                print(f"  {target}: R² = {perf['R2']:.4f}, MAE = {perf['MAE']:.3f}")

        # Check if this is the best model
        adjusted_score = test_r2 - (0.3 * generalization_gap)
        if adjusted_score > best_test_score:
            best_test_score = adjusted_score
            best_model = model
            best_model_name = name

    except Exception as e:
        print(f"Model {name} failed: {e}")
        all_model_results[name] = {'error': str(e)}

In [ ]:
# Display All Model Results

print("\n" + "="*60)
print("ALL MODELS PERFORMANCE COMPARISON")
print("="*60)

if all_model_results:
    # Create comparison table
    comparison_data = []
    for name, results in all_model_results.items():
        if 'error' not in results:
            # Determine model type
            if 'RandomForest' in name:
                model_type = 'RF'
            elif 'XGB' in name:
                model_type = 'XGB'
            elif 'Light' in name:
                model_type = 'LGB'
            elif 'ANN' in name:
                model_type = 'ANN'
            elif 'SVR' in name:
                model_type = 'SVR'
            else:
                model_type = 'Other'

            comparison_data.append({
                'Model': name,
                'Type': model_type,
                'Test R²': f"{results['test_r2']:.4f}",
                'Train R²': f"{results['train_r2']:.4f}",
                'CV R²': f"{results['cv_score']:.4f}",
                'Overfit Gap': f"{results['generalization_gap']:.4f}",
                'Time (s)': f"{results['training_time']:.2f}"
            })

    # Create DataFrame and sort
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df_sorted = comparison_df.sort_values(by='Test R²', ascending=False)

    print("\nPERFORMANCE COMPARISON TABLE (Sorted by Test R²):")
    print("-" * 80)
    print(comparison_df_sorted.to_string(index=False))
    print("-" * 80)

    # Highlight best model
    best_row = comparison_df_sorted.iloc[0]
    print(f"\nBEST MODEL: {best_row['Model']}")
    print(f"  Test R²: {best_row['Test R²']}")
    print(f"  Overfit Gap: {best_row['Overfit Gap']}")
    print(f"  Training Time: {best_row['Time (s)']} seconds")

    # Visualization
    print("\n" + "="*60)
    print("VISUALIZING MODEL PERFORMANCE")
    print("="*60)

    # Prepare data for visualization
    valid_models = [m for m in all_model_results.keys() if 'error' not in all_model_results[m]]
    test_scores = [all_model_results[m]['test_r2'] for m in valid_models]
    train_scores = [all_model_results[m]['train_r2'] for m in valid_models]

    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # 1. Test scores bar chart
    x_pos = np.arange(len(valid_models))
    colors = plt.cm.Set3(np.linspace(0, 1, len(valid_models)))

    axes[0].barh(x_pos, test_scores, color=colors, alpha=0.7, edgecolor='black')
    axes[0].set_yticks(x_pos)
    axes[0].set_yticklabels(valid_models)
    axes[0].set_xlabel('Test R² Score')
    axes[0].set_title('Model Test Performance')
    axes[0].grid(True, alpha=0.3, axis='x')

    # Add value labels
    for i, v in enumerate(test_scores):
        axes[0].text(v + 0.01, i, f'{v:.3f}', va='center')

    # 2. Train vs Test scatter
    axes[1].scatter(train_scores, test_scores, s=100, alpha=0.7)
    axes[1].plot([0, 1], [0, 1], 'r--', alpha=0.5, label='Train = Test')
    axes[1].set_xlabel('Train R² Score')
    axes[1].set_ylabel('Test R² Score')
    axes[1].set_title('Train vs Test Performance')
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()

    # Add model labels
    for i, txt in enumerate(valid_models):
        axes[1].annotate(txt, (train_scores[i], test_scores[i]),
                       xytext=(5, 5), textcoords='offset points', fontsize=9)

    plt.tight_layout()
    plt.show()

    # Show best model details
    best_model_name = best_row['Model']
    best_results = all_model_results[best_model_name]

    print(f"\nBest Model Details: {best_model_name}")
    print(f"  CV Score: {best_results['cv_score']:.4f} (±{best_results['cv_std']:.4f})")
    print(f"  Training Time: {best_results['training_time']:.2f}s")

    if 'target_performance' in best_results:
        print("\n  Target-wise Performance:")
        for target, perf in best_results['target_performance'].items():
            print(f"    {target}: R²={perf['R2']:.4f}, MAE={perf['MAE']:.3f}")

else:
    print("No models were successfully trained.")

#Hybrid model

In [ ]:
# ===============================================
# Hybrid Stacking Model with Top 3 Models
# ===============================================

print("=" * 60)
print("Building Hybrid Stacking Ensemble (Top 3 Models)")
print("=" * 60)

# Select top 3 models based on test R² scores
top_3_models = ['XGBoost', 'RandomForest_Tuned', 'ANN']
print(f"Selected top 3 models: {top_3_models}")

# Extract the models from the trained models dictionary
base_models_list = []
for model_name in top_3_models:
    if model_name in models:
        base_models_list.append((model_name, models[model_name]))
    else:
        print(f"Warning: {model_name} not found in trained models")

if len(base_models_list) < 2:
    print("Need at least 2 base models to create a hybrid ensemble.")
else:
    # Custom stacking implementation for multi-output
    class MultiOutputStacking(BaseEstimator, RegressorMixin):
        def __init__(self, base_models, meta_model, preprocessor=None, n_folds=5):
            self.base_models = base_models
            self.meta_model = meta_model
            self.preprocessor = preprocessor
            self.n_folds = n_folds
            self.base_models_ = []
            self.meta_models_ = []

        def fit(self, X, y):
            from sklearn.model_selection import KFold

            if hasattr(y, 'columns'):
                self.target_names_ = y.columns.tolist()
                n_targets = len(self.target_names_)
            else:
                self.target_names_ = None
                n_targets = y.shape[1] if len(y.shape) > 1 else 1

            self.n_targets = n_targets
            self.n_base_models = len(self.base_models)

            print("Training base models...")
            self.base_models_ = []
            for name, model in self.base_models:
                print(f"  Training: {name}")
                model_clone = clone(model)
                model_clone.fit(X, y)
                self.base_models_.append((name, model_clone))

            # Generate meta-features using cross-validation
            kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=RANDOM_STATE)
            n_samples = X.shape[0]

            meta_features = np.zeros((n_samples, self.n_base_models * n_targets))

            print(f"Generating meta-features ({self.n_folds}-fold CV)...")

            for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X), 1):
                X_train_fold = X.iloc[train_idx] if hasattr(X, 'iloc') else X[train_idx]
                X_val_fold = X.iloc[val_idx] if hasattr(X, 'iloc') else X[val_idx]
                y_train_fold = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx]

                for i, (name, _) in enumerate(self.base_models_):
                    model_fold = clone(self.base_models_[i][1])
                    model_fold.fit(X_train_fold, y_train_fold)

                    y_pred = model_fold.predict(X_val_fold)

                    if n_targets == 1:
                        meta_features[val_idx, i] = y_pred.flatten()
                    else:
                        for t in range(n_targets):
                            meta_features[val_idx, i * n_targets + t] = y_pred[:, t]

                if fold_idx % 2 == 0 or fold_idx == self.n_folds:
                    print(f"    Fold {fold_idx}/{self.n_folds} completed")

            print("Getting final base model predictions...")
            self.base_predictions_full_ = []
            for i, (name, model) in enumerate(self.base_models_):
                y_pred_full = model.predict(X)
                self.base_predictions_full_.append(y_pred_full)

            print("Processing features for meta-model...")
            X_processed = self._process_features(X)

            X_combined = np.hstack([X_processed, meta_features])

            self.meta_models_ = []
            print("Training meta-models...")

            y_meta = y.values if hasattr(y, 'values') else y

            for t in range(n_targets):
                if n_targets == 1:
                    y_target = y_meta.ravel()
                else:
                    y_target = y_meta[:, t]

                meta_model_clone = clone(self.meta_model)
                meta_model_clone.fit(X_combined, y_target)
                self.meta_models_.append(meta_model_clone)

            self.n_original_features_ = X_processed.shape[1]

            return self

        def _process_features(self, X):
            if self.preprocessor is not None:
                if not hasattr(self.preprocessor, 'transform'):
                    self.preprocessor.fit(X)

                X_processed = self.preprocessor.transform(X)

                if hasattr(X_processed, 'toarray'):
                    X_processed = X_processed.toarray()
                return X_processed
            else:
                for name, model in self.base_models_:
                    if hasattr(model, 'named_steps') and 'preprocessor' in model.named_steps:
                        self.preprocessor = model.named_steps['preprocessor']
                        X_processed = self.preprocessor.transform(X)
                        if hasattr(X_processed, 'toarray'):
                            X_processed = X_processed.toarray()
                        return X_processed

                try:
                    X_processed = X.values if hasattr(X, 'values') else X
                except:
                    X_processed = X

                return X_processed

        def predict(self, X):
            base_predictions = []
            for i, (name, model) in enumerate(self.base_models_):
                y_pred = model.predict(X)
                base_predictions.append(y_pred)

            n_samples = X.shape[0]
            meta_features = np.zeros((n_samples, self.n_base_models * self.n_targets))

            for i, preds in enumerate(base_predictions):
                if self.n_targets == 1:
                    meta_features[:, i] = preds.flatten()
                else:
                    for t in range(self.n_targets):
                        meta_features[:, i * self.n_targets + t] = preds[:, t]

            X_processed = self._process_features(X)

            X_combined = np.hstack([X_processed, meta_features])

            final_predictions = []
            for t, meta_model in enumerate(self.meta_models_):
                pred = meta_model.predict(X_combined)
                final_predictions.append(pred)

            if self.n_targets == 1:
                return np.array(final_predictions[0]).reshape(-1, 1)
            else:
                return np.column_stack(final_predictions)

        def get_feature_importance(self):
            if not hasattr(self.meta_models_[0], 'coef_'):
                return None

            all_coefs = []
            for meta_model in self.meta_models_:
                if hasattr(meta_model, 'coef_'):
                    all_coefs.append(meta_model.coef_)

            if all_coefs:
                return np.mean(all_coefs, axis=0)
            return None

        def get_params(self, deep=True):
            return {
                'base_models': self.base_models,
                'meta_model': self.meta_model,
                'preprocessor': self.preprocessor,
                'n_folds': self.n_folds
            }

        def set_params(self, **params):
            for param, value in params.items():
                setattr(self, param, value)
            return self

    # Create and train the hybrid model
    try:
        preprocessor = None
        for name, model in base_models_list:
            if hasattr(model, 'named_steps') and 'preprocessor' in model.named_steps:
                preprocessor = model.named_steps['preprocessor']
                break

        from sklearn.linear_model import Ridge
        meta_model = Ridge(alpha=0.5, random_state=RANDOM_STATE)

        hybrid_model = MultiOutputStacking(
            base_models=base_models_list,
            meta_model=meta_model,
            preprocessor=preprocessor,
            n_folds=5
        )

        print("\n" + "=" * 60)
        print("Hybrid Model Configuration:")
        print("=" * 60)
        print(f"Base models: {[name for name, _ in base_models_list]}")
        print(f"Meta-model: {meta_model.__class__.__name__} (alpha={meta_model.alpha})")
        print(f"CV folds for meta-features: {hybrid_model.n_folds}")
        print(f"Number of targets: {len(target_cols)}")

        print("\n" + "=" * 60)
        print("Training Hybrid Stacking Model")
        print("=" * 60)

        start_time = time.time()
        print("Training hybrid model...")

        hybrid_model.fit(X_train, y_train)

        training_time = time.time() - start_time
        print(f"Hybrid model training completed in {training_time:.2f} seconds")

        print("\n" + "=" * 60)
        print("Evaluating Hybrid Model Performance")
        print("=" * 60)

        y_train_pred_hybrid = hybrid_model.predict(X_train)
        y_test_pred_hybrid = hybrid_model.predict(X_test)

        train_r2_hybrid = r2_score(y_train, y_train_pred_hybrid, multioutput='uniform_average')
        test_r2_hybrid = r2_score(y_test, y_test_pred_hybrid, multioutput='uniform_average')
        train_mae_hybrid = mean_absolute_error(y_train, y_train_pred_hybrid, multioutput='uniform_average')
        test_mae_hybrid = mean_absolute_error(y_test, y_test_pred_hybrid, multioutput='uniform_average')
        generalization_gap_hybrid = train_r2_hybrid - test_r2_hybrid

        print(f"Hybrid Model Performance:")
        print(f"  Training R²: {train_r2_hybrid:.4f}")
        print(f"  Test R²: {test_r2_hybrid:.4f}")
        print(f"  Training MAE: {train_mae_hybrid:.3f}")
        print(f"  Test MAE: {test_mae_hybrid:.3f}")
        print(f"  Generalization Gap: {generalization_gap_hybrid:.4f}")
        print(f"  Training Time: {training_time:.2f}s")

        print("\nHybrid Model Target Performance:")
        target_performance_hybrid = {}
        for i, target in enumerate(target_cols):
            if i < y_test_pred_hybrid.shape[1] and target in y_test.columns:
                target_r2 = r2_score(y_test[target], y_test_pred_hybrid[:, i])
                target_mae = mean_absolute_error(y_test[target], y_test_pred_hybrid[:, i])
                target_performance_hybrid[target] = {'R2': target_r2, 'MAE': target_mae}
                print(f"  {target}: R² = {target_r2:.4f}, MAE = {target_mae:.3f}")

        best_individual_name = "XGBoost"
        best_individual_results = all_model_results[best_individual_name]
        best_individual_test_r2 = best_individual_results['test_r2']

        improvement = test_r2_hybrid - best_individual_test_r2
        improvement_percent = (improvement / abs(best_individual_test_r2)) * 100

        print(f"\nComparison with Best Individual Model ({best_individual_name}):")
        print(f"  Individual Model Test R²: {best_individual_test_r2:.4f}")
        print(f"  Hybrid Model Test R²: {test_r2_hybrid:.4f}")
        print(f"  Improvement: {improvement:.4f} ({improvement_percent:+.2f}%)")

        all_model_results['Hybrid_Top3'] = {
            'cv_score': None,
            'cv_std': None,
            'train_r2': train_r2_hybrid,
            'test_r2': test_r2_hybrid,
            'train_mae': train_mae_hybrid,
            'test_mae': test_mae_hybrid,
            'generalization_gap': generalization_gap_hybrid,
            'training_time': training_time,
            'target_performance': target_performance_hybrid,
            'model': hybrid_model,
            'improvement_over_best': improvement,
            'improvement_percent': improvement_percent
        }

        print("\n" + "=" * 60)
        print("Performance Comparison: Hybrid vs Individual Models")
        print("=" * 60)

        comparison_data = []

        for model_name in top_3_models + ['SVR', 'LightGBM']:
            if model_name in all_model_results:
                results = all_model_results[model_name]
                comparison_data.append({
                    'Model': model_name,
                    'Type': 'Individual',
                    'Test R²': results['test_r2'],
                    'Test MAE': results['target_performance']['H2']['MAE'] if 'target_performance' in results else None,
                    'Training Time': results['training_time']
                })

        comparison_data.append({
            'Model': 'Hybrid_Top3',
            'Type': 'Ensemble',
            'Test R²': test_r2_hybrid,
            'Test MAE': test_mae_hybrid,
            'Training Time': training_time
        })

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df = comparison_df.sort_values('Test R²', ascending=False)

        print("\nPerformance Comparison Table:")
        print("-" * 80)
        print(comparison_df.to_string(index=False))
        print("-" * 80)

        fig, axes = plt.subplots(1, 2, figsize=(12, 5))

        models_plot = comparison_df['Model'].values
        test_r2_plot = comparison_df['Test R²'].values

        bars1 = axes[0].bar(models_plot, test_r2_plot,
                           color=['green' if 'Hybrid' in m else 'blue' for m in models_plot],
                           alpha=0.7, edgecolor='black')
        axes[0].set_ylabel('Test R² Score')
        axes[0].set_title('Model Comparison: Test R² Score')
        axes[0].axhline(y=best_individual_test_r2, color='red', linestyle='--',
                      alpha=0.5, label=f'Best Individual ({best_individual_name})')
        axes[0].legend()
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].grid(True, alpha=0.3, axis='y')

        for bar, v in zip(bars1, test_r2_plot):
            height = bar.get_height()
            axes[0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{v:.3f}', ha='center', va='bottom', fontsize=9)

        hybrid_r2 = test_r2_hybrid
        axes[1].bar(['Best Individual', 'Hybrid Ensemble'],
                  [best_individual_test_r2, hybrid_r2],
                  color=['blue', 'green'], alpha=0.7, edgecolor='black')
        axes[1].set_ylabel('Test R² Score')
        axes[1].set_title(f'Improvement: {improvement_percent:+.2f}%')
        axes[1].grid(True, alpha=0.3, axis='y')

        if improvement > 0:
            axes[1].annotate('', xy=(1, hybrid_r2), xytext=(0, best_individual_test_r2),
                          arrowprops=dict(arrowstyle='->', color='green', lw=2))
            axes[1].text(0.5, (best_individual_test_r2 + hybrid_r2)/2,
                      f'+{improvement:.3f}', ha='center', va='center',
                      backgroundcolor='white', fontweight='bold')

        plt.tight_layout()
        plt.show()

        print("\n" + "=" * 60)
        print("Saving Hybrid Model")
        print("=" * 60)

        try:
            model_filename = os.path.join(SAVE_DIR, 'hybrid_top3_model.pkl')
            joblib.dump(hybrid_model, model_filename)

            results_filename = os.path.join(SAVE_DIR, 'model_comparison_results.csv')
            comparison_df.to_csv(results_filename, index=False)

            print(f"Hybrid model saved: {model_filename}")
            print(f"Results saved: {results_filename}")

            feature_importance = hybrid_model.get_feature_importance()
            if feature_importance is not None:
                print("\nHybrid Model Feature Importance (first 10 features):")
                n_original_features = hybrid_model.n_original_features_
                original_importance = feature_importance[:n_original_features]

                if hasattr(X, 'columns'):
                    feature_names = X.columns.tolist()
                    if len(feature_names) >= 10:
                        for i, (name, imp) in enumerate(zip(feature_names[:10], original_importance[:10])):
                            print(f"  {i+1:2d}. {name}: {imp:.4f}")

        except Exception as e:
            print(f"Could not save model: {e}")

    except Exception as e:
        print(f"Error creating/training hybrid model: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# ===============================================
# Build Species Properties Library
# ===============================================

print("=" * 60)
print("Building Species Properties Library")
print("=" * 60)

# Build enhanced species properties library
prop_keywords = ['C', 'H', 'O', 'N', 'S', 'Ash', 'Moisture', 'Volatile', 'Fixed', 'HHV', 'LHV', 'VM']
selected_props = []

for col in numeric_cols:
    col_lower = col.lower()
    if any(kw.lower() in col_lower for kw in prop_keywords):
        selected_props.append(col)

if not selected_props:
    selected_props = [col for col in numeric_cols if col not in ['Temperature (°C)', 'S/B', 'ER']][:8]

print(f"Selected properties for species library: {selected_props}")

species_col = 'TYPES'

if species_col in df_clean.columns:
    species_props = (df_clean
                     .groupby(species_col)[selected_props]
                     .agg(['mean', 'std', 'count'])
                     .round(4))

    species_props_mean = (df_clean
                         .groupby(species_col)[selected_props]
                         .mean()
                         .reset_index()
                         .rename(columns={species_col: 'species'}))

    print(f"Species library built: {species_props_mean.shape[0]} species")
    print(f"Sample species: {species_props_mean['species'].tolist()[:5]}")

    print("\nSample species properties:")
    print(species_props_mean.head())

else:
    print(f"Warning: Column '{species_col}' not found in dataset")
    print("Creating synthetic species properties...")

    unique_biomass = ["Biomass_A", "Biomass_B", "Biomass_C", "Biomass_D", "Biomass_E"]

    dummy_data = []
    for i, biomass in enumerate(unique_biomass):
        props = {}
        for prop in selected_props:
            if 'C' in prop:
                props[prop] = 40 + np.random.uniform(-10, 10)
            elif 'H' in prop:
                props[prop] = 5 + np.random.uniform(-2, 2)
            elif 'O' in prop:
                props[prop] = 45 + np.random.uniform(-10, 10)
            elif 'Ash' in prop.lower():
                props[prop] = 5 + np.random.uniform(0, 10)
            elif 'Moisture' in prop:
                props[prop] = 10 + np.random.uniform(0, 15)
            else:
                props[prop] = np.random.uniform(0, 100)

        props['species'] = biomass
        dummy_data.append(props)

    species_props_mean = pd.DataFrame(dummy_data)
    print(f"Created synthetic species library: {species_props_mean.shape[0]} species")

target_cols = ['H2', 'CO', 'CO2', 'CH4']

print("\nSpecies properties library ready")
print(f"Species count: {len(species_props_mean)}")
print(f"Properties: {len(selected_props)} features")

In [ ]:
# ===============================================
# Generalized Mixture Functions (1-4 Biomass)
# ===============================================

print("=" * 60)
print("Creating Generalized Mixture System (1-4 Biomass)")
print("=" * 60)

def build_generalized_input(biomass_types, ratios, temperature, sb, er, gasifier_type=None):
    """
    Build input features for mixture of 1-4 biomass types

    Parameters:
    -----------
    biomass_types : list of str (1 to 4 biomass types)
    ratios : list of float (mixing ratios, must sum to 1.0)
    temperature : float
    sb : float (S/B ratio)
    er : float (ER ratio)
    gasifier_type : str or None

    Returns:
    --------
    pd.DataFrame: Prepared input features
    """
    n_biomass = len(biomass_types)
    if n_biomass < 1 or n_biomass > 4:
        raise ValueError(f"Number of biomass types must be 1-4, got {n_biomass}")

    if len(ratios) != n_biomass:
        raise ValueError(f"Need {n_biomass} ratios, got {len(ratios)}")

    ratios = np.array(ratios, dtype=float)
    ratios = ratios / ratios.sum()

    biomass_props = []
    for biomass in biomass_types:
        props = species_props_mean[species_props_mean['species'] == biomass]
        if len(props) == 0:
            available_species = species_props_mean['species'].tolist()
            matches = [s for s in available_species if biomass.lower() in s.lower()]
            if matches:
                biomass = matches[0]
                props = species_props_mean[species_props_mean['species'] == biomass]

        if len(props) == 0:
            available = species_props_mean['species'].tolist()
            raise ValueError(f"Biomass '{biomass}' not found. Available: {available}")

        biomass_props.append(props.iloc[0])

    mixed_features = {}

    for prop in selected_props:
        weighted_sum = 0
        for i in range(n_biomass):
            weighted_sum += ratios[i] * biomass_props[i][prop]
        mixed_features[prop] = weighted_sum

    mixed_features['Temperature (°C)'] = temperature
    mixed_features['S/B'] = sb
    mixed_features['ER'] = er

    mixed_features['Temp_SB_interaction'] = temperature * sb
    mixed_features['Temp_ER_interaction'] = temperature * er
    mixed_features['SB_ER_interaction'] = sb * er

    if n_biomass > 1:
        diversity = -np.sum(ratios * np.log(ratios + 1e-10))
        mixed_features['Mixture_Diversity'] = diversity
        mixed_features['Dominance_Ratio'] = np.max(ratios)

    if gasifier_type is not None:
        mixed_features['Gasifier type'] = gasifier_type

    input_df = pd.DataFrame([mixed_features])

    for col in numeric_cols + categorical_cols:
        if col not in input_df.columns:
            input_df[col] = np.nan

    input_df = input_df[numeric_cols + categorical_cols]

    return input_df, ratios

def predict_generalized_mixture(biomass_types, ratios, temperature, sb, er, gasifier_type=None):
    """
    Predict syngas for mixture of 1-4 biomass types using hybrid model
    """
    if not isinstance(biomass_types, list):
        biomass_types = [biomass_types]
    if not isinstance(ratios, list):
        ratios = [ratios]

    n_biomass = len(biomass_types)

    if n_biomass == 1:
        ratios = [1.0]

    input_df, normalized_ratios = build_generalized_input(
        biomass_types, ratios, temperature, sb, er, gasifier_type
    )

    try:
        predictions = hybrid_model.predict(input_df)[0]
    except Exception as e:
        print(f"Hybrid model failed, using {best_model_name} as fallback: {e}")
        predictions = best_model.predict(input_df)[0]

    results = {
        'predicted_syngas': {
            target: float(predictions[i]) for i, target in enumerate(target_cols)
        },
        'biomass_types': biomass_types,
        'ratios': normalized_ratios.tolist(),
        'ratios_percent': [round(r*100, 1) for r in normalized_ratios],
        'temperature': temperature,
        'S/B': sb,
        'ER': er,
        'n_biomass': n_biomass
    }

    return results

def test_generalized_predictions():
    """Test function for generalized mixture predictions"""
    print("\nTesting Generalized Predictions...")

    available_species = species_props_mean['species'].tolist()
    print(f"Available species: {len(available_species)} total")

    if len(available_species) < 4:
        print(f"Only {len(available_species)} species available, creating synthetic ones...")
        for i in range(max(0, 4 - len(available_species))):
            new_species = f"Synth_Biomass_{i+1}"
            if new_species not in available_species:
                available_species.append(new_species)

    test_species = available_species[:4]
    print(f"Testing with species: {test_species}")

    test_cases = [
        ([test_species[0]], [1.0], 800, 1.0, 0.3, "Single biomass"),
        ([test_species[0], test_species[1]], [0.6, 0.4], 800, 1.0, 0.3, "2 biomass mix"),
        ([test_species[0], test_species[1], test_species[2]],
         [0.4, 0.35, 0.25], 800, 1.0, 0.3, "3 biomass mix"),
    ]

    if len(test_species) >= 4:
        test_cases.append(
            ([test_species[0], test_species[1], test_species[2], test_species[3]],
             [0.3, 0.3, 0.2, 0.2], 800, 1.0, 0.3, "4 biomass mix")
        )

    for biomass_types, ratios, temp, sb, er, desc in test_cases:
        try:
            print(f"\n{'='*40}")
            print(f"Testing: {desc}")
            print(f"Biomass: {biomass_types}")
            print(f"Ratios: {ratios}")

            result = predict_generalized_mixture(biomass_types, ratios, temp, sb, er)
            print(f"Success:")
            print(f"   Normalized Ratios: {result['ratios_percent']}%")
            print(f"   Syngas: H2={result['predicted_syngas']['H2']:.2f}%, "
                  f"CO={result['predicted_syngas']['CO']:.2f}%, "
                  f"CO2={result['predicted_syngas']['CO2']:.2f}%, "
                  f"CH4={result['predicted_syngas']['CH4']:.2f}%")
        except Exception as e:
            print(f"Failed: {e}")

# Run tests
test_generalized_predictions()
print("\nGeneralized mixture prediction functions created")

In [ ]:
# ===============================================
# Generalized Optimization (1-4 Biomass)
# ===============================================

print("=" * 60)
print("Creating Generalized Optimization System")
print("=" * 60)

def generalized_objective(x, biomass_types, objective='H2'):
    """
    Objective function for N-biomass optimization

    Parameters:
    -----------
    x : array-like
        - For 2 biomass: [temp, sb, er, r1]
        - For 3 biomass: [temp, sb, er, r1, r2]
        - For 4 biomass: [temp, sb, er, r1, r2, r3]
    biomass_types : list of str
    objective : str, gas to maximize

    Returns:
    --------
    float: Negative of target value (for minimization)
    """
    n_biomass = len(biomass_types)

    temp, sb, er = x[0], x[1], x[2]

    if n_biomass == 1:
        ratios = [1.0]
    elif n_biomass == 2:
        r1 = x[3]
        ratios = [r1, 1.0 - r1]
    elif n_biomass == 3:
        r1, r2 = x[3], x[4]
        ratios = [r1, r2, 1.0 - r1 - r2]
    elif n_biomass == 4:
        r1, r2, r3 = x[3], x[4], x[5]
        ratios = [r1, r2, r3, 1.0 - r1 - r2 - r3]

    penalty = 0

    if not (600 <= temp <= 950):
        penalty += 1000 * min(abs(temp-600), abs(temp-950))

    for ratio in ratios:
        if ratio < 0 or ratio > 1:
            penalty += 1000 * (abs(min(0, ratio)) + abs(max(0, ratio-1)))

    if sb < 0.1 or sb > 3.0:
        penalty += 1000 * min(abs(sb-0.1), abs(sb-3.0))

    if er < 0.1 or er > 2.8:
        penalty += 1000 * min(abs(er-0.1), abs(er-2.8))

    try:
        result = predict_generalized_mixture(biomass_types, ratios, temp, sb, er)
        target_value = result['predicted_syngas'].get(objective, 0)

        if n_biomass > 1:
            small_ratio_penalty = sum(100 for r in ratios if r < 0.05)
            penalty += small_ratio_penalty

        return -target_value + penalty

    except Exception as e:
        return 1e6

def optimize_generalized_mixture(biomass_types, objective='H2', max_iter=50):
    """
    Optimize conditions for mixture of 1-4 biomass types

    Parameters:
    -----------
    biomass_types : list of str (1 to 4 biomass types)
    objective : str, gas to maximize
    max_iter : int, maximum iterations

    Returns:
    --------
    dict: Optimization results
    """
    n_biomass = len(biomass_types)

    print(f"\nOptimizing {' + '.join(biomass_types)} for maximum {objective}")
    print(f"   Number of biomass types: {n_biomass}")

    bounds = [
        (600.0, 950.0),    # Temperature
        (0.1, 3.0),        # S/B ratio
        (0.1, 2.8),        # ER ratio
    ]

    if n_biomass == 2:
        bounds.append((0.0, 1.0))
    elif n_biomass == 3:
        bounds.append((0.0, 1.0))
        bounds.append((0.0, 1.0))
    elif n_biomass == 4:
        bounds.append((0.0, 1.0))
        bounds.append((0.0, 1.0))
        bounds.append((0.0, 1.0))

    try:
        result = differential_evolution(
            generalized_objective,
            bounds,
            args=(biomass_types, objective),
            strategy='best1bin',
            maxiter=max_iter,
            popsize=min(15, 5 * n_biomass),
            seed=RANDOM_STATE,
            polish=True,
            tol=0.01,
            disp=False
        )

        temp_opt, sb_opt, er_opt = result.x[0], result.x[1], result.x[2]

        if n_biomass == 1:
            ratios_opt = [1.0]
        elif n_biomass == 2:
            r1_opt = result.x[3]
            ratios_opt = [r1_opt, 1.0 - r1_opt]
        elif n_biomass == 3:
            r1_opt, r2_opt = result.x[3], result.x[4]
            ratios_opt = [r1_opt, r2_opt, 1.0 - r1_opt - r2_opt]
        elif n_biomass == 4:
            r1_opt, r2_opt, r3_opt = result.x[3], result.x[4], result.x[5]
            ratios_opt = [r1_opt, r2_opt, r3_opt, 1.0 - r1_opt - r2_opt - r3_opt]

        prediction = predict_generalized_mixture(
            biomass_types, ratios_opt, temp_opt, sb_opt, er_opt
        )

        return {
            'success': True,
            'objective': objective,
            'biomass_types': biomass_types,
            'optimal_conditions': {
                'temperature': float(temp_opt),
                'S/B': float(sb_opt),
                'ER': float(er_opt),
                'ratios': [float(r) for r in ratios_opt],
                'ratios_percent': [round(r*100, 1) for r in ratios_opt]
            },
            'predicted_syngas': prediction['predicted_syngas'],
            'objective_value': prediction['predicted_syngas'].get(objective, 0),
            'iterations': result.nit,
            'message': f'Optimization completed in {result.nit} iterations'
        }

    except Exception as e:
        return {
            'success': False,
            'message': f'Optimization failed: {str(e)}',
            'biomass_types': biomass_types,
            'objective': objective
        }

def test_generalized_optimization():
    """Test function for generalized optimization"""
    print("\nTesting Generalized Optimization...")

    available_species = species_props_mean['species'].tolist()
    if len(available_species) < 4:
        print(f"Need at least 4 species, got {len(available_species)}")
        return

    test_cases = [
        ([available_species[0]], "H2", "Single biomass"),
        ([available_species[0], available_species[1]], "CO", "2 biomass mix"),
        ([available_species[0], available_species[1], available_species[2]],
         "H2", "3 biomass mix"),
        ([available_species[0], available_species[1], available_species[2], available_species[3]],
         "CO", "4 biomass mix")
    ]

    for biomass_types, objective, desc in test_cases:
        print(f"\n{'='*40}")
        print(f"Testing: {desc}")
        print(f"Biomass: {biomass_types}")
        print(f"Objective: Maximize {objective}")

        result = optimize_generalized_mixture(biomass_types, objective, max_iter=30)

        if result['success']:
            print(f"Optimization Successful")
            opt = result['optimal_conditions']
            print(f"   Temperature: {opt['temperature']:.1f}°C")
            print(f"   S/B: {opt['S/B']:.3f}")
            print(f"   ER: {opt['ER']:.3f}")
            print(f"   Ratios: {opt['ratios_percent']}%")
            print(f"   Max {objective}: {result['objective_value']:.2f}%")
        else:
            print(f"Optimization Failed: {result['message']}")

# Run optimization tests
test_generalized_optimization()
print("\nGeneralized optimization functions created")

## Creating Gradio Interface

In [ ]:
# ===============================================
# Enhanced Gradio Interface for Biomass Gasification
# ===============================================

print("=" * 60)
print("Creating Enhanced Gradio Interface")
print("=" * 60)

import gradio as gr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import json
from datetime import datetime

# Initialize session state
class SessionState:
    def __init__(self):
        self.current_prediction = None
        self.optimization_history = []
        self.species_data = species_props_mean.copy()
        self.custom_species = []

    def get_available_species(self):
        all_species = list(self.species_data['species'].values) + self.custom_species
        return sorted(all_species)

session_state = SessionState()

# Visualization Functions
def create_syngas_bar_chart(predictions, title="Syngas Composition"):
    gases = list(predictions.keys())
    values = list(predictions.values())

    fig = go.Figure(data=[
        go.Bar(x=gases, y=values, marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ])

    fig.update_layout(
        title=title,
        xaxis_title="Gas Component",
        yaxis_title="Percentage (%)",
        yaxis_range=[0, 100],
        template="plotly_white",
        height=400
    )

    return fig

def create_parameter_effects_plot(biomass_types, ratios, base_temp=800, base_sb=1.0, base_er=0.3):
    temps = np.linspace(600, 950, 10)
    sbs = np.linspace(0.1, 3.0, 10)

    gases = ['H2', 'CO', 'CO2', 'CH4']
    temp_data = {gas: [] for gas in gases}
    sb_data = {gas: [] for gas in gases}

    for temp in temps:
        try:
            pred = predict_generalized_mixture(biomass_types, ratios, temp, base_sb, base_er)
            for gas in gases:
                temp_data[gas].append(pred['predicted_syngas'][gas])
        except:
            for gas in gases:
                temp_data[gas].append(0)

    for sb in sbs:
        try:
            pred = predict_generalized_mixture(biomass_types, ratios, base_temp, sb, base_er)
            for gas in gases:
                sb_data[gas].append(pred['predicted_syngas'][gas])
        except:
            for gas in gases:
                sb_data[gas].append(0)

    temp_fig = go.Figure()
    sb_fig = go.Figure()

    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    labels = ['H₂', 'CO', 'CO₂', 'CH₄']

    for i, gas in enumerate(gases):
        temp_fig.add_trace(
            go.Scatter(
                x=temps,
                y=temp_data[gas],
                mode='lines+markers',
                name=labels[i],
                line=dict(color=colors[i], width=2.5),
                marker=dict(size=8)
            )
        )

        sb_fig.add_trace(
            go.Scatter(
                x=sbs,
                y=sb_data[gas],
                mode='lines+markers',
                name=labels[i],
                line=dict(color=colors[i], width=2.5),
                marker=dict(size=8),
                showlegend=False
            )
        )

    temp_fig.update_layout(
        title="Temperature Effects on Gas Composition",
        xaxis_title="Temperature (°C)",
        yaxis_title="Percentage (%)",
        template="plotly_white",
        height=400,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    sb_fig.update_layout(
        title="S/B Ratio Effects on Gas Composition",
        xaxis_title="S/B Ratio",
        yaxis_title="Percentage (%)",
        template="plotly_white",
        height=400
    )

    combined_fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Temperature Effects", "S/B Ratio Effects"),
        horizontal_spacing=0.15
    )

    for trace in temp_fig.data:
        combined_fig.add_trace(trace, row=1, col=1)

    for trace in sb_fig.data:
        combined_fig.add_trace(trace, row=1, col=2)

    combined_fig.update_layout(
        height=450,
        template="plotly_white",
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=1.15,
            xanchor="center",
            x=0.5,
            orientation="h"
        )
    )

    combined_fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
    combined_fig.update_xaxes(title_text="S/B Ratio", row=1, col=2)
    combined_fig.update_yaxes(title_text="Percentage (%)", row=1, col=1)
    combined_fig.update_yaxes(title_text="Percentage (%)", row=1, col=2)

    return combined_fig

def create_mixture_visualization(syngas, biomass_types, ratios_percent):
    if len(biomass_types) > 1:
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=("Syngas Composition", "Biomass Mixture Ratio"),
            specs=[[{"type": "bar"}, {"type": "pie"}]]
        )

        gases = list(syngas.keys())
        values = list(syngas.values())

        fig.add_trace(
            go.Bar(x=gases, y=values, marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']),
            row=1, col=1
        )

        fig.add_trace(
            go.Pie(labels=biomass_types, values=ratios_percent, hole=0.3),
            row=1, col=2
        )

        fig.update_layout(
            height=400,
            template="plotly_white",
            showlegend=True
        )

        fig.update_xaxes(title_text="Gas Component", row=1, col=1)
        fig.update_yaxes(title_text="Percentage (%)", row=1, col=1)

    else:
        fig = create_syngas_bar_chart(syngas, f"Single Biomass: {biomass_types[0]}")

    return fig

def create_optimization_visualization(syngas, biomass_types, ratios_percent, objective, objective_value):
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=("Syngas Composition", "Biomass Mixture", "Component Comparison"),
        specs=[[{"type": "bar"}, {"type": "pie"}, {"type": "bar"}]]
    )

    gases = ['H2', 'CO', 'CO2', 'CH4']
    values = [syngas.get(gas, 0) for gas in gases]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    labels = ['H₂', 'CO', 'CO₂', 'CH₄']

    fig.add_trace(
        go.Bar(x=labels, y=values, marker_color=colors),
        row=1, col=1
    )

    if len(biomass_types) > 1:
        fig.add_trace(
            go.Pie(labels=biomass_types, values=ratios_percent, hole=0.3),
            row=1, col=2
        )
    else:
        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=100,
                title={"text": f"Single Biomass<br>{biomass_types[0]}"},
                domain={'row': 0, 'column': 1}
            ),
            row=1, col=2
        )

    fig.add_trace(
        go.Bar(x=labels, y=values, marker_color=colors, showlegend=False),
        row=1, col=3
    )

    fig.update_layout(
        height=400,
        template="plotly_white",
        title_text=f"Optimization Results: Maximizing {objective} = {objective_value:.2f}%",
        title_x=0.5
    )

    fig.update_xaxes(title_text="Gas Component", row=1, col=1)
    fig.update_yaxes(title_text="Percentage (%)", row=1, col=1)
    fig.update_yaxes(title_text="Percentage (%)", row=1, col=3)

    return fig

# Prediction Functions
def format_prediction_results(prediction_data):
    results_text = f"""
Prediction Results:
-------------------
Biomass Types: {', '.join(prediction_data['biomass_types'])}
Number of Components: {prediction_data['n_biomass']}
Ratios: {prediction_data['ratios_percent']}%

Process Conditions:
-------------------
Temperature: {prediction_data['temperature']} °C
S/B Ratio: {prediction_data['S/B']:.3f}
ER Ratio: {prediction_data['ER']:.3f}

Syngas Composition:
-------------------
H₂: {prediction_data['predicted_syngas']['H2']:.2f}%
CO: {prediction_data['predicted_syngas']['CO']:.2f}%
CO₂: {prediction_data['predicted_syngas']['CO2']:.2f}%
CH₄: {prediction_data['predicted_syngas']['CH4']:.2f}%
"""
    return results_text

def predict_single_biomass(biomass_type, temperature, sb_ratio, er_ratio, gasifier_type):
    try:
        result = predict_generalized_mixture(
            [biomass_type], [1.0], temperature, sb_ratio, er_ratio, gasifier_type
        )

        session_state.current_prediction = result

        syngas_chart = create_syngas_bar_chart(result['predicted_syngas'], f"Single Biomass: {biomass_type}")
        results_text = format_prediction_results(result)
        param_plot = create_parameter_effects_plot([biomass_type], [1.0], temperature, sb_ratio, er_ratio)

        return syngas_chart, results_text, param_plot, result

    except Exception as e:
        error_msg = f"Prediction failed: {str(e)}"
        return None, error_msg, None, None

def predict_multi_biomass(num_components_str, biomass1, biomass2, biomass3, biomass4,
                         ratio1, ratio2, ratio3, ratio4,
                         temperature, sb_ratio, er_ratio, gasifier_type):
    try:
        num_components = int(num_components_str)

        # Collect biomass types and ratios
        biomass_types = []
        ratios_percent = []

        if num_components >= 1:
            biomass_types.append(biomass1)
            ratios_percent.append(float(ratio1))
        if num_components >= 2:
            biomass_types.append(biomass2)
            ratios_percent.append(float(ratio2))
        if num_components >= 3:
            biomass_types.append(biomass3)
            ratios_percent.append(float(ratio3))
        if num_components >= 4:
            biomass_types.append(biomass4)
            ratios_percent.append(float(ratio4))

        # Convert ratios to fractions
        ratios = [r/100 for r in ratios_percent]

        # Make prediction
        result = predict_generalized_mixture(
            biomass_types, ratios, float(temperature), float(sb_ratio), float(er_ratio), gasifier_type
        )

        session_state.current_prediction = result

        # Create visualizations
        syngas_chart = create_syngas_bar_chart(result['predicted_syngas'],
                                              f"Mixture of {num_components} Biomass")
        mixture_chart = create_mixture_visualization(result['predicted_syngas'],
                                                    biomass_types, result['ratios_percent'])
        param_plot = create_parameter_effects_plot(biomass_types, ratios,
                                                  float(temperature), float(sb_ratio), float(er_ratio))
        results_text = format_prediction_results(result)

        info_text = f"**Mixture Composition:**\n"
        for i, (bio, ratio) in enumerate(zip(biomass_types, result['ratios_percent']), 1):
            info_text += f"- Biomass {i}: {bio} ({ratio}%)\n"

        return syngas_chart, mixture_chart, param_plot, results_text, info_text, result

    except Exception as e:
        error_msg = f"Prediction failed: {str(e)}"
        return None, None, None, error_msg, "", None

# Optimization Functions
def optimize_multi_biomass(num_components_str, biomass1, biomass2, biomass3, biomass4,
                          objective_gas, max_iterations):
    try:
        num_components = int(num_components_str)

        # Collect biomass types
        biomass_types = []
        if num_components >= 1:
            biomass_types.append(biomass1)
        if num_components >= 2:
            biomass_types.append(biomass2)
        if num_components >= 3:
            biomass_types.append(biomass3)
        if num_components >= 4:
            biomass_types.append(biomass4)

        # Run optimization
        result = optimize_generalized_mixture(
            biomass_types, objective_gas, int(max_iterations)
        )

        if result['success']:
            session_state.optimization_history.append(result)

            opt = result['optimal_conditions']
            syngas = result['predicted_syngas']

            results_text = f"""
Optimization Results:
====================
Target: Maximize {objective_gas}
Biomass Types: {', '.join(result['biomass_types'])}

Optimal Conditions:
-------------------
Temperature: {opt['temperature']:.1f} °C
S/B Ratio: {opt['S/B']:.3f}
ER Ratio: {opt['ER']:.3f}
Ratios: {opt['ratios_percent']}%

Optimal Syngas:
---------------
H₂: {syngas['H2']:.2f}%
CO: {syngas['CO']:.2f}%
CO₂: {syngas['CO2']:.2f}%
CH₄: {syngas['CH4']:.2f}%

Maximum {objective_gas}: {result['objective_value']:.2f}%
Iterations: {result['iterations']}
"""

            opt_chart = create_optimization_visualization(
                syngas, biomass_types, opt['ratios_percent'],
                objective_gas, result['objective_value']
            )

            param_plot = create_parameter_effects_plot(
                biomass_types, opt['ratios'],
                opt['temperature'], opt['S/B'], opt['ER']
            )

            return results_text, opt_chart, param_plot, result

        else:
            error_msg = f"Optimization failed: {result['message']}"
            return error_msg, None, None, None

    except Exception as e:
        error_msg = f"Optimization error: {str(e)}"
        return error_msg, None, None, None

# Create the Gradio Interface
def create_enhanced_interface():
    available_species = session_state.get_available_species()
    if not available_species:
        available_species = ["Type-C", "Type-I", "Type-M", "Type-S"]

    with gr.Blocks(title="Biomass Gasification Optimizer", theme=gr.themes.Soft()) as interface:
        gr.Markdown("# Biomass Gasification Optimization System")
        gr.Markdown("Predict and optimize syngas production from biomass mixtures")

        with gr.Tabs():
            # Tab 1: Single Biomass Prediction
            with gr.TabItem("Single Biomass"):
                with gr.Row():
                    with gr.Column(scale=1):
                        biomass_single = gr.Dropdown(
                            choices=available_species,
                            value=available_species[0],
                            label="Select Biomass Type"
                        )

                        temperature_single = gr.Slider(
                            minimum=600, maximum=950, value=800, step=10,
                            label="Temperature (°C)"
                        )

                        sb_ratio_single = gr.Slider(
                            minimum=0.1, maximum=3.0, value=1.0, step=0.1,
                            label="S/B Ratio"
                        )

                        er_ratio_single = gr.Slider(
                            minimum=0.1, maximum=2.8, value=0.3, step=0.1,
                            label="ER Ratio"
                        )

                        gasifier_single = gr.Radio(
                            choices=["fix", "fluid"], value="fix", label="Gasifier Type"
                        )

                        predict_single_btn = gr.Button("Predict", variant="primary")

                    with gr.Column(scale=2):
                        single_syngas_chart = gr.Plot(label="Syngas Composition")
                        single_results = gr.Textbox(label="Results", lines=15)
                        single_param_plot = gr.Plot(label="Parameter Effects")

                predict_single_btn.click(
                    predict_single_biomass,
                    inputs=[biomass_single, temperature_single, sb_ratio_single, er_ratio_single, gasifier_single],
                    outputs=[single_syngas_chart, single_results, single_param_plot, gr.State()]
                )

            # Tab 2: Multi-Biomass Mixture
            with gr.TabItem("Multi-Biomass Mixture"):
                with gr.Row():
                    with gr.Column(scale=1):
                        num_components = gr.Dropdown(
                            choices=["1", "2", "3", "4"],
                            value="2",
                            label="Number of Biomass Components"
                        )

                        # Biomass dropdowns
                        biomass1 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[0],
                            label="Biomass Type 1",
                            visible=True
                        )

                        biomass2 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[1 % len(available_species)],
                            label="Biomass Type 2",
                            visible=True
                        )

                        biomass3 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[2 % len(available_species)] if len(available_species) > 2 else available_species[0],
                            label="Biomass Type 3",
                            visible=False
                        )

                        biomass4 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[3 % len(available_species)] if len(available_species) > 3 else available_species[0],
                            label="Biomass Type 4",
                            visible=False
                        )

                        # Ratio inputs - using Number components instead of Sliders to avoid min=max issue
                        ratio1 = gr.Number(
                            value=100,
                            label="Ratio Biomass 1 (%)",
                            visible=True,
                            minimum=0,
                            maximum=100
                        )

                        ratio2 = gr.Number(
                            value=50,
                            label="Ratio Biomass 2 (%)",
                            visible=True,
                            minimum=0,
                            maximum=100
                        )

                        ratio3 = gr.Number(
                            value=33,
                            label="Ratio Biomass 3 (%)",
                            visible=False,
                            minimum=0,
                            maximum=100
                        )

                        ratio4 = gr.Number(
                            value=25,
                            label="Ratio Biomass 4 (%)",
                            visible=False,
                            minimum=0,
                            maximum=100
                        )

                        temperature_mix = gr.Slider(
                            minimum=600, maximum=950, value=800, step=10,
                            label="Temperature (°C)"
                        )

                        sb_ratio_mix = gr.Slider(
                            minimum=0.1, maximum=3.0, value=1.0, step=0.1,
                            label="S/B Ratio"
                        )

                        er_ratio_mix = gr.Slider(
                            minimum=0.1, maximum=2.8, value=0.3, step=0.1,
                            label="ER Ratio"
                        )

                        gasifier_mix = gr.Radio(
                            choices=["fix", "fluid"], value="fix", label="Gasifier Type"
                        )

                        predict_mix_btn = gr.Button("Predict Mixture", variant="primary")

                    with gr.Column(scale=2):
                        mix_syngas_chart = gr.Plot(label="Syngas Composition")
                        mix_mixture_chart = gr.Plot(label="Mixture Visualization")
                        mix_param_plot = gr.Plot(label="Parameter Effects")
                        mix_results = gr.Textbox(label="Results", lines=10)
                        mix_info = gr.Markdown(label="Mixture Details")

                def update_multi_components(num_components):
                    num_components = int(num_components)

                    # Update biomass dropdown visibility
                    biomass_updates = [
                        gr.update(visible=True),  # biomass1 always visible
                        gr.update(visible=num_components >= 2),
                        gr.update(visible=num_components >= 3),
                        gr.update(visible=num_components >= 4)
                    ]

                    # Update ratio inputs
                    ratio_updates = []
                    for i in range(4):
                        if i < num_components:
                            if i == 0 and num_components == 1:
                                # Single biomass: fixed 100%
                                ratio_updates.append(gr.update(
                                    visible=True,
                                    value=100,
                                    minimum=100,
                                    maximum=100,
                                    label="Ratio Biomass 1 (%)"
                                ))
                            else:
                                # Multiple biomass: adjustable
                                default_value = 100 / num_components
                                ratio_updates.append(gr.update(
                                    visible=True,
                                    value=default_value,
                                    minimum=0,
                                    maximum=100,
                                    label=f"Ratio Biomass {i+1} (%)"
                                ))
                        else:
                            # Hide unused inputs
                            ratio_updates.append(gr.update(visible=False))

                    return biomass_updates + ratio_updates

                num_components.change(
                    update_multi_components,
                    inputs=[num_components],
                    outputs=[biomass1, biomass2, biomass3, biomass4,
                            ratio1, ratio2, ratio3, ratio4]
                )

                predict_mix_btn.click(
                    predict_multi_biomass,
                    inputs=[num_components, biomass1, biomass2, biomass3, biomass4,
                           ratio1, ratio2, ratio3, ratio4,
                           temperature_mix, sb_ratio_mix, er_ratio_mix, gasifier_mix],
                    outputs=[mix_syngas_chart, mix_mixture_chart, mix_param_plot,
                            mix_results, mix_info, gr.State()]
                )

            # Tab 3: Optimization
            with gr.TabItem("Optimization"):
                with gr.Row():
                    with gr.Column(scale=1):
                        opt_num_components = gr.Dropdown(
                            choices=["1", "2", "3", "4"],
                            value="2",
                            label="Number of Biomass Components"
                        )

                        opt_biomass1 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[0],
                            label="Biomass Type 1",
                            visible=True
                        )

                        opt_biomass2 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[1 % len(available_species)],
                            label="Biomass Type 2",
                            visible=True
                        )

                        opt_biomass3 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[2 % len(available_species)] if len(available_species) > 2 else available_species[0],
                            label="Biomass Type 3",
                            visible=False
                        )

                        opt_biomass4 = gr.Dropdown(
                            choices=available_species,
                            value=available_species[3 % len(available_species)] if len(available_species) > 3 else available_species[0],
                            label="Biomass Type 4",
                            visible=False
                        )

                        objective_gas = gr.Dropdown(
                            choices=["H2", "CO", "CO2", "CH4"],
                            value="H2",
                            label="Gas to Maximize"
                        )

                        max_iterations = gr.Slider(
                            minimum=10, maximum=100, value=30, step=5,
                            label="Maximum Iterations"
                        )

                        optimize_btn = gr.Button("Run Optimization", variant="primary")

                    with gr.Column(scale=2):
                        opt_results = gr.Textbox(label="Optimization Results", lines=20)
                        opt_chart = gr.Plot(label="Optimal Conditions")
                        opt_param_plot = gr.Plot(label="Parameter Effects at Optimum")

                def update_opt_components(num_components):
                    num_components = int(num_components)
                    return [
                        gr.update(visible=True),  # biomass1 always visible
                        gr.update(visible=num_components >= 2),
                        gr.update(visible=num_components >= 3),
                        gr.update(visible=num_components >= 4)
                    ]

                opt_num_components.change(
                    update_opt_components,
                    inputs=[opt_num_components],
                    outputs=[opt_biomass2, opt_biomass3, opt_biomass4]
                )

                optimize_btn.click(
                    optimize_multi_biomass,
                    inputs=[opt_num_components, opt_biomass1, opt_biomass2, opt_biomass3, opt_biomass4,
                           objective_gas, max_iterations],
                    outputs=[opt_results, opt_chart, opt_param_plot, gr.State()]
                )

            # Tab 4: Sensitivity Analysis
            with gr.TabItem("Sensitivity Analysis"):
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### Parameter Sensitivity Analysis")
                        gr.Markdown("Analyze how different parameters affect syngas composition")

                        sa_biomass = gr.Dropdown(
                            choices=available_species,
                            value=available_species[0],
                            label="Select Biomass Type"
                        )

                        sa_temperature_range = gr.Slider(
                            minimum=600, maximum=950, value=800, step=10,
                            label="Base Temperature (°C)"
                        )

                        sa_sb_range = gr.Slider(
                            minimum=0.1, maximum=3.0, value=1.0, step=0.1,
                            label="Base S/B Ratio"
                        )

                        sa_er_range = gr.Slider(
                            minimum=0.1, maximum=2.8, value=0.3, step=0.1,
                            label="Base ER Ratio"
                        )

                        analyze_btn = gr.Button("Analyze Sensitivity", variant="primary")

                    with gr.Column():
                        sa_plot = gr.Plot(label="Sensitivity Analysis")
                        sa_insights = gr.Markdown(label="Key Insights")

                def analyze_sensitivity(biomass, temp, sb, er):
                    try:
                        param_plot = create_parameter_effects_plot([biomass], [1.0], temp, sb, er)

                        insights = f"""
                        **Sensitivity Analysis for {biomass}:**

                        **Temperature Range (600-950°C):**
                        - Higher temperatures generally increase H₂ production
                        - Optimal temperature range: 800-900°C
                        - Above 900°C, CO production may decrease

                        **S/B Ratio Range (0.1-3.0):**
                        - Moderate S/B ratios (1.0-2.0) optimize syngas quality
                        - Low S/B ratios favor CO production
                        - High S/B ratios favor H₂ production

                        **Current Settings:**
                        - Temperature: {temp}°C
                        - S/B Ratio: {sb}
                        - ER Ratio: {er}

                        **Recommendations:**
                        1. For maximum H₂: Use higher temperature (850-900°C) with S/B ≈ 1.5
                        2. For maximum CO: Use moderate temperature (750-800°C) with S/B ≈ 0.8
                        3. For balanced syngas: Use temperature ≈ 800°C with S/B ≈ 1.0
                        """

                        return param_plot, insights

                    except Exception as e:
                        error_msg = f"Sensitivity analysis failed: {str(e)}"
                        return None, error_msg

                analyze_btn.click(
                    analyze_sensitivity,
                    inputs=[sa_biomass, sa_temperature_range, sa_sb_range, sa_er_range],
                    outputs=[sa_plot, sa_insights]
                )

            # Tab 5: Model Information
            with gr.TabItem("Model Information"):
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### Hybrid Model Architecture")
                        model_architecture = gr.Markdown("""
                        **Ensemble Model Components:**

                        1. **XGBoost (Extreme Gradient Boosting)**
                           - Trees: 500
                           - Max depth: 8
                           - Learning rate: 0.05

                        2. **Random Forest**
                           - Trees: 200
                           - Max depth: 15
                           - Features: 70%

                        3. **Artificial Neural Network**
                           - Layers: 128-64-32
                           - Activation: ReLU
                           - Optimizer: Adam

                        4. **Support Vector Regression**
                           - Kernel: RBF
                           - C: 10.0
                           - Epsilon: 0.01

                        **Stacking Strategy:**
                        - Base models trained on full dataset
                        - Meta-model (Ridge Regression) learns to combine predictions
                        - 5-fold cross-validation for meta-features
                        """)

                    with gr.Column():
                        gr.Markdown("### Feature Importance")
                        feature_info = gr.Markdown("""
                        **Key Predictive Features:**

                        1. **Biomass Properties:**
                           - Carbon content (C)
                           - Hydrogen content (H)
                           - Oxygen content (O)
                           - Ash content
                           - Moisture

                        2. **Process Parameters:**
                           - Temperature
                           - Steam-to-Biomass ratio (S/B)
                           - Equivalence ratio (ER)

                        3. **Interaction Terms:**
                           - Temperature × S/B
                           - Temperature × ER
                           - Mixture diversity index

                        **Target Variables:**
                        - H₂ (Hydrogen)
                        - CO (Carbon Monoxide)
                        - CO₂ (Carbon Dioxide)
                        - CH₄ (Methane)

                        **Model Accuracy:**
                        - Overall R²: > 0.95
                        - H₂ prediction R²: > 0.97
                        - CO prediction R²: > 0.96
                        """)

        gr.Markdown("---")
        gr.Markdown("**Biomass Gasification Optimization System** | Research Tool")

    return interface

print("=" * 60)
print("Launching Enhanced Interface")
print("=" * 60)

# Create and launch interface
interface = create_enhanced_interface()

print("\nInterface created successfully!")
print("\nTo launch the interface:")
print("interface.launch(share=True)  # For public sharing")
print("interface.launch()            # For local use")

In [ ]:
interface.launch(share=True)